# TLS 1.3 handshake overview
This is the basic TLS 1.3 handshake:

<img src="images/handshake_tls13.png" alt="Handshake TLS 1.3" width="400"/>

In [ ]:
from scapy.all import *
load_layer('tls')

In [ ]:
record1_str = open('raw_data/tls_session_13/01_cli.raw', 'rb').read()
record1 = TLS(record1_str)
sess = record1.tls_session
record1.show()

In [ ]:
record2_str = open('raw_data/tls_session_13/02_srv.raw', 'rb').read()
record2 = TLS(record2_str, tls_session=sess.mirror())
record2.show()

In [ ]:
record3_str = open('raw_data/tls_session_13/03_cli.raw', 'rb').read()
record3 = TLS(record3_str, tls_session=sess.mirror())
record3.show()

In [ ]:
# The PFS relies on the ECDH secret below being kept from observers, and deleted right after the key exchange
#from cryptography.hazmat.primitives.asymmetric.ec import EllipticCurvePrivateNumbers
#from cryptography.hazmat.backends import default_backend
#secp256r1_client_privkey = open('raw_data/tls_session_13/cli_key.raw', 'rb').read()
#pubnum = sess.tls13_client_pubshares["secp256r1"].public_numbers()
#privnum = EllipticCurvePrivateNumbers(pkcs_os2ip(secp256r1_client_privkey), pubnum)
#privkey = privnum.private_key(default_backend())
#sess.tls13_client_privshares["secp256r1"] = privkey

In [ ]:
record4_str = open('raw_data/tls_session_13/04_srv.raw', 'rb').read()
record4 = TLS(record4_str, tls_session=sess.mirror())
record4.show()

In [ ]:
record5_str = open('raw_data/tls_session_13/05_srv.raw', 'rb').read()
record5 = TLS(record5_str, tls_session=sess)
record5.show()

In [ ]:
record6_str = open('raw_data/tls_session_13/06_cli.raw', 'rb').read()
record6 = TLS(record6_str, tls_session=sess.mirror())
record6.show()

## Observations sur TLS 1.3
* Certificat désormais chiffré...
* ...mais pas le Server Name dans le ClientHello
* Risques du mode 0-RTT